In [30]:
import requests
import base64
import io
import os
import pandas as pd
import cv2
import configparser
from tqdm import tqdm
from roboflow import Roboflow
config = configparser.ConfigParser()
config.read("../config.ini")

rf = Roboflow(api_key="CeyqMPgn8RZNppm9JpjD")

In [6]:
PROCESSED_DATA_PATH = config["PATHS"]["PROCESSED_DATA_PATH"]
print(PROCESSED_DATA_PATH)
print(METADATA_PATH)

/media/my_ftp/BasesDeDatos_Torax_RX_CT/COVID19_CT/processed
/media/my_ftp/BasesDeDatos_Torax_RX_CT/COVID19_CT/processed/segmentation/metadata.csv


In [56]:
experiment_folder = os.path.join(PROCESSED_DATA_PATH,"object_detection", "experiment")
processed_metadata_df = pd.read_pickle(os.path.join(PROCESSED_DATA_PATH, "object_detection", "metadata_with_bboxes.pkl"))

print(processed_metadata_df.shape)
print(processed_metadata_df.groupby(["split"]).size())

(87799, 50)
split
test     17560
train    70239
dtype: int64


In [71]:
def roboflow_uploader(img_paths, split=None, batch=None):
    print(rf.workspace())    # Specify the project for upload
    project = rf.workspace("covid19-detection").project("tfm-muit")
    print("Uploading....")
    for img in tqdm(img_paths):
        project.upload(img)

def upload_images_to_roboflow(img_paths, split=None, batch=None):
    # Copy the images to an intermediate folder and subsitute the file name by a number to anonymize the data, then delete that folder
    intermediate_folder = os.path.join(experiment_folder, "intermediate")
    print(intermediate_folder)
    os.makedirs(intermediate_folder, exist_ok=True)
    images_to_upload = []
    print("Copying images to intermediate folder")
    for i, img_path in enumerate(tqdm(img_paths)):
        #img = cv2.imread(img_path)
        dest_path = os.path.join(intermediate_folder, f"{i}.png")
        #cv2.imwrite(dest_path, img)
        images_to_upload.append(dest_path)
    roboflow_uploader(images_to_upload, split, batch)
    os.rmdir(intermediate_folder)
                        
def images_to_upload_selector(df, num_imgs=100, random=True, enhanced=False):
    filterded_df = processed_metadata_df[processed_metadata_df["split"] == "train"]
    filterded_df = filterded_df[filterded_df["has_infection"] == True]
    sample_df = filterded_df.sample(num_imgs)
    if enhanced:
        sample_df["ct_slice_path"] = sample_df["ct_slice_path"].apply(lambda x: x.replace("rgb_images", "enhanced"))
    img_paths = sample_df["ct_slice_path"].tolist()
    return img_paths, sample_df

In [72]:
images_to_upload, exp_df = images_to_upload_selector(processed_metadata_df, num_imgs=100, random=True, enhanced=False)

In [73]:
upload_images_to_roboflow(images_to_upload, batch=2)
exp_df.to_pickle(os.path.join(experiment_folder, "roboflow_upload_1.pkl"))

/media/my_ftp/BasesDeDatos_Torax_RX_CT/COVID19_CT/processed/object_detection/experiment/intermediate
Copying images to intermediate folder


  0%|          | 0/100 [00:00<?, ?it/s]

100%|██████████| 100/100 [00:42<00:00,  2.37it/s]

loading Roboflow workspace...


{
  "name": "Covid19 Detection",
  "url": "covid19-detection",
  "projects": [
    "covid19-detection/hard-hat-sample-rjmj4",
    "covid19-detection/tfm-muit"
  ]
}
loading Roboflow workspace...
loading Roboflow project...
Uploading....


100%|██████████| 100/100 [02:06<00:00,  1.27s/it]


OSError: [Errno 39] Directory not empty: '/media/my_ftp/BasesDeDatos_Torax_RX_CT/COVID19_CT/processed/object_detection/experiment/intermediate'

In [ ]:
# visualize 10 uploaded images along with the annotations (annotations are in the exp_df dataframe and the column name should be parameterized)
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np

def visualize_annotations(img_path, df, column_name="bboxes"):
    img = cv2.imread(img_path)
    fig, ax = plt.subplots(1)
    ax.imshow(img)
    for _, row in df.iterrows():
        xmin, ymin, xmax, ymax = row[column_name]
        rect = patches.Rectangle((xmin, ymin), xmax-xmin, ymax-ymin, linewidth=1, edgecolor='r', facecolor='none')
        ax.add_patch(rect)
    plt.show()

sample_exp_df = exp_df.sample(20)

fig, axs = plt.subplots(4, 5, figsize=(15, 12))
axs = axs.flatten()

for i, (_, row) in enumerate(sample_exp_df.iterrows()):
    img_path = row["ct_slice_path"]
    visualize_annotations(img_path, sample_exp_df, column_name="bboxes")
    axs[i].imshow(cv2.imread(img_path))
    axs[i].axis('off')

plt.tight_layout()
plt.show()
